In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
from scipy.signal import argrelextrema
from deep_sort_realtime.deepsort_tracker import DeepSort

# 필요한 상수 및 설정
CONFIDENCE_THRESHOLD = 0.4
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
RED = (0, 0, 255)
YELLOW = (0, 255, 255)
MIN_BOX_AREA = 5000  # 최소 박스 면적 설정

# 클래스 리스트 로드 함수
def load_class_list(file_path):
    with open(file_path, 'r') as f:
        return f.read().split('\n')

# 모델 초기화 함수
def initialize_yolo(model_path):
    model = YOLO(model_path)
    return model

# 이미지 파일 목록 가져오기 함수
def get_image_files(image_folder_path):
    file_names = os.listdir(image_folder_path)
    image_files = [f for f in file_names if f.endswith(('.jpg', '.png'))]
    image_files.sort()
    return image_files

# IOU 계산 함수
def calculate_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    return inter_area / (box1_area + box2_area - inter_area) if (box1_area + box2_area - inter_area) > 0 else 0

# 포함 비율 계산 함수
def calculate_containment_ratio(outer_box, inner_box):
    x1_min, y1_min, x1_max, y1_max = outer_box
    x2_min, y2_min, x2_max, y2_max = inner_box

    inner_area = (x2_max - x2_min) * (y2_max - y2_min)
    outer_area = (x1_max - x1_min) * (y1_max - y1_min)

    if outer_area > 0:
        containment_ratio = inner_area / outer_area
        return containment_ratio >= 0.90
    return False

# 바운딩 박스 그룹화 함수
def group_boxes(boxes, labels, confidences, iou_min=0.02, iou_max=0.13):
    groups = []
    used = set()

    for i, box in enumerate(boxes):
        if i in used or labels[i] != 0:
            continue
        group = [(box, confidences[i], labels[i])]
        used.add(i)

        blinkers = []
        for j, other_box in enumerate(boxes):
            if j != i and j not in used and labels[j] == 1:
                iou = calculate_iou(box, other_box)
                if iou_min <= iou <= iou_max or calculate_containment_ratio(box, other_box):
                    blinkers.append((other_box, confidences[j], labels[j]))
                    used.add(j)

        blinkers = blinkers[:2]
        group.extend(blinkers)

        groups.append(group)

    return groups

# 라벨 업데이트 함수 수정
def update_labels(grouped_boxes):
    updated_boxes = []

    for idx, group in enumerate(grouped_boxes):
        car_label = f"car{idx + 1}"

        sorted_group = sorted(group, key=lambda item: item[0][0])

        updated_group = []
        if len(sorted_group) == 3:
            new_labels = [f"{car_label}-1", f"{car_label}-L", f"{car_label}-R"]
        else:
            new_labels = [f"{car_label}-{j + 1}" for j in range(len(sorted_group))]

        for (box, conf, original_label), new_label in zip(sorted_group, new_labels):
            updated_group.append({
                'original_label': int(original_label),
                'new_label': new_label,
                'box': box,
                'confidence': conf
            })

        updated_boxes.append(updated_group)

    return updated_boxes

# 함수 A: 라벨 수가 3개인 그룹 처리
def A(group, img, idx):
    # 함수 A의 내용...
    # 이전과 동일하게 유지
    # 결과 출력 부분 수정
    print(f"프레임 {idx + 1}: 방향지시등 켜짐 - 속도를 줄이시오")

# 함수 B: 라벨 수가 2개인 그룹 처리
def B(group, frames, idx):
    # 함수 B의 내용...
    # 결과 출력 부분 수정
    print(f"프레임 {idx + 1}: 우회전 - 속도를 줄이시오")

# 함수 C: 라벨 수가 1개인 그룹 처리
def C(group, frames, idx):
    # 함수 C의 내용...
    # 결과 출력 부분 수정
    print(f"프레임 {idx + 1}: 주행 - 속도를 유지하시오")

# 메인 함수
def main():
    IMAGE_FOLDER_PATH = '/Users/kimyu/Desktop/trainn/test4/test4-2'
    MODEL_PATH = '/Users/kimyu/Desktop/융캡디/yun/best.pt'
    CLASS_LIST_PATH = '/Users/kimyu/Desktop/융캡디/yun/yolo_class.txt'

    class_list = load_class_list(CLASS_LIST_PATH)
    model = initialize_yolo(MODEL_PATH)
    image_files = get_image_files(IMAGE_FOLDER_PATH)
    frames = []

    processed_results = []

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(IMAGE_FOLDER_PATH, image_file)
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Could not load image {image_path}")
            continue

        frames.append(img)

        # 모델 예측
        result = model.predict(source=[img], save=False, verbose=False)[0]

        if result.orig_img is not None and result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            labels = result.boxes.cls.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()

            if len(boxes) == 0:
                print(f"No detections in image {idx}")
                continue

            # 바운딩 박스 그룹화 및 라벨 업데이트
            grouped_boxes = group_boxes(boxes, labels, confidences)
            updated_grouped_boxes = update_labels(grouped_boxes)

            # 그룹화된 결과 저장
            processed_results.append(updated_grouped_boxes)

            # 각 그룹에 대해 라벨 수에 따라 함수 호출
            for group in updated_grouped_boxes:
                num_labels = len(group)
                if num_labels == 1:
                    C(group, frames, idx)
                elif num_labels == 2:
                    B(group, frames, idx)
                elif num_labels == 3:
                    A(group, img, idx)
                else:
                    print(f"프레임 {idx + 1}: 감지된 그룹이 없습니다.")
        else:
            processed_results.append([])

if __name__ == "__main__":
    main()



0: 384x640 2 Cars, 4 Lights, 169.8ms
Speed: 1.5ms preprocess, 169.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 169.2ms
Speed: 1.4ms preprocess, 169.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 180.0ms
Speed: 1.3ms preprocess, 180.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 5 Lights, 172.9ms
Speed: 1.3ms preprocess, 172.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 5 Lights, 162.2ms
Speed: 1.4ms preprocess, 162.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 161.3ms
Speed: 1.3ms preprocess, 161.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 5 Lights, 149.1ms
Speed: 1.6ms preprocess, 149.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 5 Lights, 151.6ms
Speed: 1.5